# Encoding


## ASCII

ASCII es un estándar de codificación de 7 bits que permite la representación de texto utilizando números enteros del 0 al 127.

Usando la siguiente matriz de enteros, convierta los números a sus caracteres ASCII correspondientes para obtener una bandera.

```
[99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]
```

> En Python, la función `chr()` se puede utilizar para convertir un número ordinal ASCII en un carácter (la función `ord()` hace lo contrario).




In [ ]:
array = [99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]

print("".join(chr(c) for c in array))

## Hex

Cuando ciframos algo, el texto cifrado resultante suele tener bytes que no son caracteres ASCII imprimibles. Si queremos compartir nuestros datos cifrados, es común codificarlos en algo más fácil de usar y portátil en diferentes sistemas.

El hexadecimal se puede utilizar de tal manera para representar cadenas ASCII.
Primero, cada letra se convierte en un número ordinal según la tabla ASCII (como en el desafío anterior). Luego, los números decimales se convierten a números de base 16, también conocidos como hexadecimales. Los números se pueden combinar entre sí, en una larga cadena hexadecimal.

A continuación se incluye una bandera codificada como una cadena hexadecimal.
Decodifica esto nuevamente en bytes para obtener la bandera.

```
63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d
```

> En Python, la función `bytes.fromhex()` se puede utilizar para convertir hexadecimal a bytes. Se puede invocar el método de instancia `.hex()` en cadenas de bytes para obtener la representación hexadecimal.

Recursos:
  - [ASCII table](https://www.rapidtables.com/code/text/ascii-table.html)
  - [Wikipedia: Hexadecimal (Español)](https://es.wikipedia.org/wiki/Sistema_hexadecimal)

In [ ]:
hex = "63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d"
print(bytes.fromhex(hex))

## Base64

Otro esquema de codificación común es Base64, que nos permite representar datos binarios como una cadena ASCII utilizando un alfabeto de 64 caracteres. Un carácter de una cadena Base64 codifica 6 dígitos binarios (bits), por lo que 4 caracteres de Base64 codifican tres bytes de 8 bits.

Base64 se usa más comúnmente en línea, por lo que los datos binarios, como imágenes, se pueden incluir fácilmente en archivos HTML o CSS.

Tome la siguiente cadena hexadecimal, decodifíquela en bytes y luego codifíquela en Base64.

```
72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf
```

> En Python, después de importar el módulo base64 con import base64, puede usar la función base64.b64encode(). Recuerde decodificar el hexadecimal primero como lo indica la descripción del desafío.



In [ ]:
import base64 as base64
hex = "72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf"
print(base64.b64encode(bytes.fromhex(hex)))

## Bytes and Big Integers

Los sistemas criptográficos como RSA funcionan con números, pero los mensajes están compuestos de caracteres. ¿Cómo debemos convertir nuestros mensajes en números para poder aplicar operaciones matemáticas?

La forma más común es tomar los bytes ordinales del mensaje, convertirlos en hexadecimales y concatenarlos. Esto se puede interpretar como un número de base 16/hexadecimal y también se puede representar en base 10/decimal.

Para ilustrarlo:
```
message: HELLO
ascii bytes: [72, 69, 76, 76, 79]
hex bytes: [0x48, 0x45, 0x4c, 0x4c, 0x4f]
base-16: 0x48454c4c4f
base-10: 310400273487
```

> La biblioteca PyCryptodome de Python implementa esto con los métodos `bytes_to_long()` y `long_to_bytes()`. Primero deberá instalar PyCryptodome e importarlo con `from Crypto.Util.number import *`. Para obtener más detalles, consulte las [preguntas frecuentes](https://cryptohack.org/faq/#install).

Convierte el siguiente entero nuevamente en un mensaje:

> 11515195063862318899931685488813747395775516287289682636499965282714637259206269





In [ ]:
!pip install pycryptodome
from Crypto.Util.number import *

big_int = 11515195063862318899931685488813747395775516287289682636499965282714637259206269
print(long_to_bytes(big_int))

## Encoding Challenge

Ahora que ya conoces las distintas codificaciones que encontrarás, veamos cómo automatizarlas.

¿Puedes superar los 100 niveles para obtener la bandera?

El archivo `13377.py` adjunto a continuación es el código fuente de lo que se está ejecutando en el servidor. El archivo `pwntools_example.py` proporciona el inicio de una solución.

Para obtener más información sobre cómo conectarse a desafíos interactivos, consulta las [preguntas frecuentes](https://cryptohack.org/faq/#netcat). ¡No dudes en pasar directamente a la criptografía si no estás de humor para un desafío de codificación!

> Si desea ejecutar y probar el desafío localmente, consulte las preguntas frecuentes para descargar el módulo `utils.listener`.

Conéctese en `socket.cryptohack.org 13377`

Archivos de desafío:
- [13377.py](https://cryptohack.org/static/challenges/13377_86793614535c47dea371d2f0e406dbd9.py)
- [pwntools_example.py](https://cryptohack.org/static/challenges/pwntools_example_f93ca6ccef2def755aa8f6d9aa6e9c5b.py)





In [ ]:
#!/usr/bin/env python3
!pip install pycryptodome
!pip install utils
from Crypto.Util.number import bytes_to_long, long_to_bytes
from utils import listener # this is cryptohack's server-side module and not part of python
import base64
import codecs
import random

FLAG = "crypto{????????????????????}"
ENCODINGS = [
    "base64",
    "hex",
    "rot13",
    "bigint",
    "utf-8",
]
with open('/usr/share/dict/words') as f:
    WORDS = [line.strip().replace("'", "") for line in f.readlines()]


class Challenge():
    def __init__(self):
        self.no_prompt = True # Immediately send data from the server without waiting for user input
        self.challenge_words = ""
        self.stage = 0

    def create_level(self):
        self.stage += 1
        self.challenge_words = "_".join(random.choices(WORDS, k=3))
        encoding = random.choice(ENCODINGS)

        if encoding == "base64":
            encoded = base64.b64encode(self.challenge_words.encode()).decode() # wow so encode
        elif encoding == "hex":
            encoded = self.challenge_words.encode().hex()
        elif encoding == "rot13":
            encoded = codecs.encode(self.challenge_words, 'rot_13')
        elif encoding == "bigint":
            encoded = hex(bytes_to_long(self.challenge_words.encode()))
        elif encoding == "utf-8":
            encoded = [ord(b) for b in self.challenge_words]

        return {"type": encoding, "encoded": encoded}

    #
    # This challenge function is called on your input, which must be JSON
    # encoded
    #
    def challenge(self, your_input):
        if self.stage == 0:
            return self.create_level()
        elif self.stage == 100:
            self.exit = True
            return {"flag": FLAG}

        if self.challenge_words == your_input["decoded"]:
            return self.create_level()

        return {"error": "Decoding fail"}


import builtins; builtins.Challenge = Challenge # hack to enable challenge to be run locally, see https://cryptohack.org/faq/#listener
listener.start_server(port=13377)

# XOR

## XOR Starter

XOR es un operador bit a bit que devuelve 0 si los bits son iguales y 1 en caso contrario. En los libros de texto, el operador XOR se denota con ⊕, pero en la mayoría de los desafíos y lenguajes de programación verá que se utiliza el signo de intercalación `^` en su lugar.

Para números binarios más largos, realizamos la operación XOR bit a bit: `0110 ^ 1010 = 1100`. Podemos realizar la operación XOR de números enteros convirtiendo primero el número entero de decimal a binario. Podemos realizar la operación XOR de cadenas convirtiendo primero cada carácter en el número entero que representa el carácter Unicode.

<table width="360px">
  <tr align="center">
    <th>A</th>
    <th>B</th>
    <th>Salida</th>
  </tr>
  <tr align="center">
    <td>0</td><td>0</td><td>0</td>
  </tr>
  <tr align="center">
    <td>0</td><td>1</td><td>1</td>
  </tr>
  <tr align="center">
    <td>1</td><td>0</td><td>1</td>
  </tr>
  <tr align="center">
    <td>1</td><td>1</td><td>0</td>
  </tr>
</table>

Dada la cadena `label`, realizamos la operación XOR de cada carácter con el número entero `13`. Convertimos estos números enteros nuevamente en una cadena y enviamos la bandera como `crypto{new_string}`.

> La biblioteca `pwntools` de Python tiene una función `xor()` muy útil que puede combinar datos de distintos tipos y longitudes mediante la operación XOR. Pero primero, es posible que quieras implementar tu propia función para resolver este problema.




In [ ]:
print("crypto{" + "".join(chr(ord(c)^13) for c in "label") + "}")

## XOR Properties

En el último desafío, viste cómo funcionaba XOR a nivel de bits. En este, cubriremos las propiedades de la operación XOR y luego las usaremos para deshacer una cadena de operaciones que han cifrado una bandera. Tener una idea de cómo funciona esto te ayudará mucho cuando ataque sistemas criptográficos reales más adelante, especialmente en la categoría de cifrados de bloques.

Hay cuatro propiedades principales que debemos considerar cuando resolvemos desafíos usando el operador XOR
```
Commutative: A ⊕ B = B ⊕ A
Associative: A ⊕ (B ⊕ C) = (A ⊕ B) ⊕ C
Identity: A ⊕ 0 = A
Self-Inverse: A ⊕ A = 0
```
Vamos a desglosar esto. Conmutativo significa que el orden de las operaciones XOR no es importante. Asociativo significa que una cadena de operaciones se puede llevar a cabo sin orden (no tenemos que preocuparnos por los corchetes). La identidad es 0, por lo que XOR con 0 "no hace nada" y, por último, algo XOR'd consigo mismo devuelve cero.

¡Pongamos esto en práctica! A continuación se muestra una serie de resultados donde se han realizado XOR'd de tres claves aleatorias juntas y con el indicador. Utilice las propiedades anteriores para deshacer el cifrado en la línea final para obtener el indicador.
```
KEY1 = a6c8b6733c9b22de7bc0253266a3867df55acde8635e19c73313
KEY2 ^ KEY1 = 37dcb292030faa90d07eec17e3b1c6d8daf94c35d4c9191a5e1e
KEY2 ^ KEY3 = c1545756687e7573db23aa1c3452a098b71a7fbf0fddddde5fc1
FLAG ^ KEY1 ^ KEY3 ^ KEY2 = 04ee9855208a2cd59091d04767ae47963170d1660df7f56f5faf
```
> Antes de realizar XOR en estos objetos, asegúrese de decodificarlos de hexadecimal a bytes.

In [ ]:
KEY1 = bytes.fromhex("a6c8b6733c9b22de7bc0253266a3867df55acde8635e19c73313")
KEY21 = bytes.fromhex("37dcb292030faa90d07eec17e3b1c6d8daf94c35d4c9191a5e1e")
KEY2 = bytes( x ^ y for x,y in zip(KEY1, KEY21))
KEY23 = bytes.fromhex("c1545756687e7573db23aa1c3452a098b71a7fbf0fddddde5fc1")
KEY3 = bytes( x ^ y for x,y in zip(KEY2, KEY23))
KEY123 = bytes.fromhex("04ee9855208a2cd59091d04767ae47963170d1660df7f56f5faf")
FLAG = bytes( x ^ y ^ w ^ z for x,y,w,z in zip(KEY1, KEY2, KEY3, KEY123))
print(FLAG)

## Favourite byte

Para los próximos desafíos, usarás lo que acabas de aprender para resolver algunos acertijos XOR más.

He ocultado algunos datos usando XOR con un solo byte, pero ese byte es un secreto. No olvides decodificar primero desde hexadecimal.
```
73626960647f6b206821204f21254f7d694f7624662065622127234f726927756d
```

In [ ]:
msg = bytes.fromhex("73626960647f6b206821204f21254f7d694f7624662065622127234f726927756d")

def decode(msg):
  for key in range(256):
    try:
        msg_decode = bytes([b ^ key for b in msg]).decode('utf-8')
        if msg_decode.startswith("crypto{") and msg_decode.endswith("}"):
            return msg_decode
    except UnicodeDecodeError:
        continue
  return "Not found"

print(decode(msg))

## You either know, XOR you don't

He cifrado la bandera con mi clave secreta, nunca podrás adivinarla.

> ¡Recuerda el formato de la bandera y cómo podría ayudarte en este desafío!

```
0e0b213f26041e480b26217f27342e175d0e070a3c5b103e2526217f27342e175d0e077e263451150104
```





In [ ]:
msg = bytes.fromhex("0e0b213f26041e480b26217f27342e175d0e070a3c5b103e2526217f27342e175d0e077e263451150104")

begin_flag = (ord(x) for x in "crypto{")
begin_key = "".join(chr(x^y) for x,y in zip(msg, begin_flag))
end_key = chr(msg[-1] ^ ord("}"))
key = begin_key + end_key
key = (key * (len(msg) // len(key) + 1))[:len(msg)]
flag = "".join(chr(x^ord(y)) for x,y in zip(msg, key))

print(flag)

## Lemur XOR

¡He escondido dos imágenes geniales de XOR con la misma clave secreta para que no puedas verlas!

> Este desafío requiere realizar un XOR visual entre los bytes RGB de las dos imágenes, no un XOR de todos los bytes de datos de los archivos.

Archivos de desafío:
* [lemur.png](https://cryptohack.org/static/challenges/lemur_ed66878c338e662d3473f0d98eedbd0d.png)
* [flag.png](https://cryptohack.org/static/challenges/flag_7ae18c704272532658c10b5faad06d74.png)

In [ ]:
from PIL import Image
from io import BytesIO
import requests

def xor_images():
    img1 = Image.open(BytesIO(requests.get("https://cryptohack.org/static/challenges/lemur_ed66878c338e662d3473f0d98eedbd0d.png").content))
    img2 = Image.open(BytesIO(requests.get("https://cryptohack.org/static/challenges/flag_7ae18c704272532658c10b5faad06d74.png").content))

    xor_img = Image.new('RGB', img1.size)

    pixeles1 = img1.load()
    pixeles2 = img2.load()
    xor_pixeles = xor_img.load()

    for x in range(img1.width):
        for y in range(img1.height):
            r1, g1, b1 = pixeles1[x, y]
            r2, g2, b2 = pixeles2[x, y]

            r_xor = r1 ^ r2
            g_xor = g1 ^ g2
            b_xor = b1 ^ b2

            xor_pixeles[x, y] = (r_xor, g_xor, b_xor)

    return xor_img

xor_images()

# Mathematics

## Greatest Common Divisor

El máximo común divisor (MCD), a veces conocido como máximo común divisor, es el número más grande que divide a dos números enteros positivos `(a,b)`.

Para `a=12, b=8` podemos calcular los divisores de `a: {1,2,3,4,6,12}` y los divisores de `b: {1,2,4,8}`. Comparando estos dos, vemos que `mcd⁡(a,b)=4`.

Ahora imaginemos que tomamos `a=11, b=17`. Tanto `a` como `b` son números primos. Como un número primo solo tiene a sí mismo y al `1` como divisores, `mcd⁡(a,b)=1`.

Decimos que para dos números enteros cualesquiera `a, b`, si `mcd⁡(a,b)=1` entonces `a` y `b` son números enteros coprimos.

Si `a` y `b` son primos, también son coprimos. Si `a` es primo y `b<a` entonces `a` y `b` son coprimos.

> Pensemos en el caso de `a` primo y `b>a`, ¿por qué no son necesariamente coprimos?

Existen muchas herramientas para calcular el MCD de dos números enteros, pero para esta tarea recomendamos consultar el [Algoritmo de Euclides](https://es.wikipedia.org/wiki/Algoritmo_de_Euclides).

Intenta codificarlo; son solo un par de líneas. Usa `a=12, b=8` para probarlo.

Ahora calcula `mcd⁡(a,b)` para `a=66528, b=52920` e introdúcelo a continuación.

In [ ]:
def mcd(a, b):
  return b if a%b == 0 else mcd(b, a%b)

mcd(66528, 52920)

## Extended GCD

Sean `a` y `b` números enteros positivos.

El algoritmo euclidiano extendido es una forma eficiente de encontrar números enteros `u, v` tales que:
```
a⋅u+b⋅v=mcd(a,b)
```

> Más adelante, cuando aprendamos a descifrar textos cifrados RSA, necesitaremos este algoritmo para calcular el inverso modular del exponente público.

Usando los dos primos `p=26513,q=32321`, encuentre los números enteros `u, v` tales que:
```
p⋅u+q⋅v=gcd(p,q)
```

Ingrese el número más bajo entre `u` y `v` como bandera.

> Sabiendo que `p, q` son primos, ¿qué esperarías que fuera `mcd(p,q)`? Para obtener más detalles sobre el algoritmo euclidiano extendido, consulte esta [página](https://web.archive.org/web/20230511143526/http://www-math.ucdenver.edu/~wcherowi/courses/m5410/exeucalg.html).

In [ ]:
def mcd_extend(a, b):
  if a == 0:
    return b, 0, 1

  gcd, x1, y1 = mcd_extend(b % a, a)
  x = y1 - (b // a) * x1
  y = x1
  return gcd, x, y

mcd_extend(26513, 32321)[2]

## Modular Arithmetic 1


Imagínese que se inclina y mira el cuaderno de un criptógrafo. Ves algunas notas al margen:
```
4 + 9 = 1
5 - 7 = 10
2 + 3 = 5
```
770 / 5.000
Al principio, podrías pensar que se han vuelto locos. Quizá por eso hay tantas filtraciones de datos hoy en día, pero esto no es más que aritmética modular módulo 12 (aunque con una notación un tanto descuidada).

Puede que no lo hayas llamado aritmética modular, pero has estado haciendo este tipo de cálculos desde que aprendiste a decir la hora (mira de nuevo esas ecuaciones y piensa en sumar horas).

Formalmente, "calcular el tiempo" se describe mediante la teoría de congruencias. Decimos que dos números enteros son congruentes módulo m si `a ≡ b mod m`.

Otra forma de decirlo es que cuando dividimos el número entero `a` por `m`, el resto es `b`. Esto te dice que si `m` divide `a` (esto se puede escribir como `m ∣ a`), entonces `a ≡ 0 mod m`.

Calcular los siguientes números enteros:
```
11 ≡ x mod 6
8146798528947 ≡ y mod 17
```
La solución es el menor de los dos números enteros, `(x,y)`, que obtuviste después de reducir por el módulo.





In [ ]:
11 % 6 if 11 % 6 < 8146798528947 % 17 else 8146798528947 % 17

## Modular Arithmetic 2

Continuaremos con el último desafío e imaginaremos que hemos elegido un módulo `p` y nos limitaremos al caso en el que `p` sea primo.

Los números enteros módulo `p` definen un campo, denominado `Fp`.

> Si el módulo no es primo, el conjunto de números enteros módulo `n` define un anillo.

Un cuerpo finito `Fp` es el conjunto de los números enteros `0,1,...,p-1`, y tanto en la suma como en la multiplicación hay elementos inversos `b+` y `b*` para cada elemento `a` en el conjunto, tal que `a+b+=0` y `a⋅b*=1`.

> ¡Tenga en cuenta que el elemento de identidad para la suma y la multiplicación es diferente! Esto se debe a que la identidad cuando se actúa con el operador no debe hacer nada: `a+0=a` y `a⋅1=a`.

Digamos que elegimos `p=17`. Calcule `3^7 mod 17`. Ahora haz lo mismo pero con `5^17 mod 17`.

¿Qué esperaría obtener por `7^16 mod 17`? Intenta calcular eso.

Este interesante hecho se conoce como el pequeño teorema de Fermat. Necesitaremos esto (y sus generalizaciones) cuando analicemos la criptografía RSA.

Ahora toma el primo `p=65537`. Calcular `273246787654^65536 mod 65537`.

¿Necesitabas una calculadora?

In [ ]:
def is_prime(n):
  if n <= 1:
    return False
  if n <= 3:
    return True
  if n % 2 == 0 or n % 3 == 0:
    return False

  i = 5
  while i * i <= n:
    if n % i == 0 or n % (i + 2) == 0:
      return False
    i += 6

  return True

def fermat(a, p):
  if(not is_prime(p)):
    raise ValueError("El número p debe ser un número primo.")

  return pow(a, p-1, p)

fermat(273246787654, 65537)

## Modular Inverting

Como hemos visto, podemos trabajar dentro de un campo finito `Fp`, sumando y multiplicando elementos, y obtener siempre un elemento más del campo.

Para todos los elementos `g` en el campo, existe un entero único `d` tal que `g ⋅ d ≡ 1 mod p`.

Este es el inverso multiplicativo de `g`.

Ejemplo: `7 ⋅ 8 = 56 ≡ 1 mod 11`

¿Cuál es el elemento inverso: `d=3−1` tal que `3 ⋅ d ≡ 1 mod 13`?

In [ ]:
next((n for n in range(1, 100) if (3 * n) % 13 == 1), None)

# Data Formats

## Privacy-Enhanced Mail?

Como hemos visto en la sección de codificación, la criptografía implica tratar datos en una amplia variedad de formatos: enteros grandes, bytes sin formato, cadenas hexadecimales y más. Se han estandarizado algunos formatos estructurados para ayudar a enviar y recibir datos criptográficos. Es útil poder reconocer y manipular estos formatos de datos comunes.

```
-----BEGIN RSA PUBLIC KEY-----
MIIBCgKC... (un montón de base64)
-----END RSA PUBLIC KEY-----
```

Envuelve los datos codificados en base64 mediante un encabezado y un pie de página de una línea para indicar cómo analizar los datos que contienen. Quizás inesperadamente, es importante que haya la cantidad correcta de guiones en el encabezado y pie de página; de lo contrario, las herramientas criptográficas no podrán reconocer el archivo.

> Hay dos enfoques principales para resolver este desafío. Los datos del certificado se pueden leer con la herramienta de línea de comandos openssl o en Python usando PyCryptodome. Recomendamos usar PyCryptodome: primero importe el módulo RSA desde `Crypto.PublicKey import RSA` y podrá leer los datos clave usando `RSA.importKey()`.

Archivos de desafío:
* [privacy_enhanced_mail.pem](https://cryptohack.org/static/challenges/privacy_enhanced_mail_1f696c053d76a78c2c531bb013a92d4a.pem)

Recursos:
* [ASN.1 vs DER vs PEM vs x509 vs PKCS#7 vs ....](https://www.cryptologie.net/article/260/asn1-vs-der-vs-pem-vs-x509-vs-pkcs7-vs/)
* [A Warm Welcome to ASN.1 and DER](https://letsencrypt.org/docs/a-warm-welcome-to-asn1-and-der/)



In [ ]:
!pip install pycryptodome

import requests
from Crypto.PublicKey import RSA

key_data = requests.get("https://cryptohack.org/static/challenges/privacy_enhanced_mail_1f696c053d76a78c2c531bb013a92d4a.pem").content

key = RSA.import_key(key_data)

key.d